In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2024-06-17 17:47:22--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.207, 74.125.135.207, 74.125.142.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   106MB/s    in 0.8s    

2024-06-17 17:47:23 (106 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [10]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

local_weights = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(
    input_shape = (150, 150, 3),
    include_top = False,
    weights = None
)

pre_trained_model.load_weights(local_weights)

for layer in pre_trained_model.layers:
  layer.trainable = False

In [11]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 74, 74, 32)           864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 74, 74, 32)           96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 74, 74, 32)           0         ['batch_normalizati

In [12]:
last_layer = pre_trained_model.get_layer('mixed7')
print("shape of last layer", last_layer.output_shape)
last_output = last_layer.output

shape of last layer (None, 7, 7, 768)


In [19]:
from tensorflow.keras import Model

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation = 'sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 74, 74, 32)           864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 74, 74, 32)           96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 74, 74, 32)           0         ['batch_normalization_94

In [20]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss = 'binary_crossentropy',
    optimizer = RMSprop(learning_rate = 0.0001),
    metrics = ['accuracy']
)

In [21]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

--2024-06-17 18:02:54--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 74.125.20.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   106MB/s    in 1.3s    

2024-06-17 18:02:56 (106 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]

--2024-06-17 18:02:56--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 74.125.20.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M

In [29]:
import zipfile

zip_ref = zipfile.ZipFile('./horse-or-human.zip', 'r')
zip_ref.extractall('tmp/horse-or-human')

zip_ref = zipfile.ZipFile('./validation-horse-or-human.zip', 'r')
zip_ref.extractall('tmp/validation-horse-or-human')

zip_ref.close()

In [35]:
import os

base_dir = 'tmp'

train_dir = os.path.join(base_dir, 'horse-or-human')
valid_dir = os.path.join(base_dir, 'validation-horse-or-human')

train_horse_dir = os.path.join(train_dir, 'horses')
train_human_dir = os.path.join(train_dir, 'humans')

valid_horse_dir = os.path.join(valid_dir, 'horses')
valid_human_dir = os.path.join(valid_dir, 'humans')

In [37]:
print("Total no training images:", len(os.listdir(train_horse_dir)) + len(os.listdir(train_human_dir)))
print("Total no validation images:", len(os.listdir(valid_horse_dir)) + len(os.listdir(valid_human_dir)))

Total no training images: 1027
Total no validation images: 256


In [42]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

valid_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'binary'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size = (150, 150),
    batch_size = 32,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [44]:
history = model.fit(
    train_generator,
    steps_per_epoch = 8,
    epochs = 20,
    verbose = 1,
    validation_data = valid_generator,
    validation_steps = 8
)

Epoch 1/20
8/8 [==============================] - 73s 9s/step - loss: 0.3309 - accuracy: 0.9155 - val_loss: 0.0280 - val_accuracy: 0.9883
Epoch 2/20
8/8 [==============================] - 70s 8s/step - loss: 0.0565 - accuracy: 0.9800 - val_loss: 0.0124 - val_accuracy: 0.9961
Epoch 3/20
8/8 [==============================] - 69s 8s/step - loss: 0.0291 - accuracy: 0.9933 - val_loss: 0.0161 - val_accuracy: 0.9922
Epoch 4/20
8/8 [==============================] - 73s 9s/step - loss: 0.0302 - accuracy: 0.9900 - val_loss: 0.0099 - val_accuracy: 0.9961
Epoch 5/20
8/8 [==============================] - 79s 10s/step - loss: 0.0237 - accuracy: 0.9900 - val_loss: 0.0283 - val_accuracy: 0.9922
Epoch 6/20
8/8 [==============================] - 69s 10s/step - loss: 0.0117 - accuracy: 0.9989 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 7/20
8/8 [==============================] - 72s 9s/step - loss: 0.0257 - accuracy: 0.9911 - val_loss: 0.0119 - val_accuracy: 1.0000
Epoch 8/20
8/8 [================